# Training a model on a standalone tabular dataset
Example of making a standalone dataset available for training a fastai deep learning application.

In this notebook we'll go through the steps to train a model on the Kuala Lumpur property dataset: https://www.kaggle.com/dragonduck/property-listings-in-kuala-lumpur



In [1]:
# imports for notebook boilerplate
!pip install -Uqq fastbook
import fastbook
from fastbook import *
from fastai.tabular.all import *


In [2]:
# imports for this notebook
import re

In [3]:
# set up the notebook for fast.ai
fastbook.setup_book()

# Ingest the dataset

The following cells assume that you have completed the following steps:
- Download data_kaggle.csv.zip from https://www.kaggle.com/dragonduck/property-listings-in-kuala-lumpur
- Unzip the downloaded file to extract data_kaggle.csv
- In your Gradient environment, create the folder /storage/archive/kl_property
- Upload data_kaggle.csv to /storage/archive/kl_property


In [4]:
# define a target path for this house price dataset
path = URLs.path('kl_property')

In [5]:
Path.cwd()

Path('/notebooks/fastai2/Deep-Learning-with-fastai-Cookbook/ch3')

In [6]:
# ingest the dataset into a Pandas dataframe
df_train = pd.read_csv(path/'data_kaggle.csv')

In [7]:
df_train.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,"KLCC, Kuala Lumpur","RM 1,250,000",2+1,3.0,2.0,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,"Damansara Heights, Kuala Lumpur","RM 6,800,000",6,7.0,NaN,Bungalow,Land area : 6900 sq. ft.,Partly Furnished
2,"Dutamas, Kuala Lumpur","RM 1,030,000",3,4.0,2.0,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
3,"Cheras, Kuala Lumpur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bukit Jalil, Kuala Lumpur","RM 900,000",4+1,3.0,2.0,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished


In [8]:
df_train.shape

(53883, 8)

In [9]:
# control whether the dependent variable is continuous or categorical
# if this switch is set to True then the values in the Price column are replaced with 
# string indicators: 0 if Price is less or equal to average; 1 if Price is above average
categorical_target = True

# Preprocessing to clean up the dataset
Unlike some other datasets featured on Kaggle, this dataset has many interesting anomalies that need to be cleaned up before fastai data preparations can be appplied to it. In particular, the Size column has values that were entered free form, which means that it needs a lot of work. For this column we've added processing to get a useful numerical value from the columns values where it's possible, but for values that are difficult to parse, we drop the row. We lose about 1% of the rows in this way - a reasonable tradeoff to make to keep the cleanup code as simple as possible.

Here are the issues that need to be corrected with this dataset:
- Price column has some misisng values. We need to remove these values
- Price column includes the ringgit symbol (the symbol for the Malaysian currency). We need to remove this symbol so that this column can be treated as a continuous column
- Size column needs to be split to into columns, one with the size type and the other with size (area)
- Size (area) column needs to update to remove the measure ("sq. ft.") and to convert area vectors into scalars
- deal with Size entries like: "5700 sf sq. ft.", "646sf~1001sf sq. ft." - remove the rows with ranges or constructs like "22&#8217;x100&#8217; sq. ft.", as well as rows that contain strings that cannot be converted into numerics




In [10]:
# function to remove the currency symbol
def remove_currency(currency_string, input_string):
    output_string = re.sub(currency_string,'',input_string)
    return(output_string)
    

In [11]:
# function to remove everything after the space in a string
def remove_after_space(input_string):
    # remove leading and trailing spaces
    input_string = input_string.strip()
    #print('input:', input_string)
    # remove everything after internal spaces
    output_string = re.sub(r'\s* .*', '', input_string)
    output_string = re.sub(r'\([^)]*\)','',output_string)
    #print('output:',output_string)
    return(output_string)

In [12]:
df_train.shape

(53883, 8)

# Clean up the Price column
- remove rows where the Price column has missing values
- remove currency symbol
- convert value to numeric

In [13]:
# remove rows with missing Price values
df_train.dropna(subset=['Price'], inplace=True)
# remove currency symbol from remaining rows
df_train['Price'] = df_train['Price'].apply(lambda x: remove_currency("RM ",x))


# convert Price column to float
df_train['Price'] = pd.to_numeric(df_train['Price'].str.replace(',',''), errors='coerce')
df_train.head()


,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,"KLCC, Kuala Lumpur",1250000,2+1,3.0,2.0,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,"Damansara Heights, Kuala Lumpur",6800000,6,7.0,NaN,Bungalow,Land area : 6900 sq. ft.,Partly Furnished
2,"Dutamas, Kuala Lumpur",1030000,3,4.0,2.0,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
4,"Bukit Jalil, Kuala Lumpur",900000,4+1,3.0,2.0,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished
5,"Taman Tun Dr Ismail, Kuala Lumpur",5350000,4+2,5.0,4.0,Bungalow,Land area : 7200 sq. ft.,Partly Furnished


In [14]:
# count how many Price values are very small
df_train[df_train.Price <= 10000].shape[0]

398

# Clean up the Size column
- split the dimension values from the size type values
- remove records with no numeric values
- remove records with problematic values
- for records with dimensions, replace the dimension (e.g. "24 x 68") with the area value (e.g.  24*28)

In [15]:
df_train['Size'].head(10)

0        Built-up : 1,335 sq. ft.
1        Land area : 6900 sq. ft.
2        Built-up : 1,875 sq. ft.
4        Built-up : 1,513 sq. ft.
5        Land area : 7200 sq. ft.
7        Land area : 3600 sq. ft.
8       Land area : 25x75 sq. ft.
9          Built-up : 904 sq. ft.
11    Land area : 22 x 80 sq. ft.
12       Land area : 1900 sq. ft.
Name: Size, dtype: object

In [16]:
df_train.shape

(53635, 8)

In [17]:
# drop rows with missing values in the Size column
df_train.dropna(subset=['Size'], inplace=True)

In [18]:
#show examples of rows where the Size column has no digits
df_train[~df_train.Size.str.contains(r'\d')].head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
996,"Desa ParkCity, Kuala Lumpur",3880000,5+1,6.0,NaN,3-sty Terrace/Link House,Land area : Kuala Lumpur sq. ft.,Partly Furnished
1006,"Desa ParkCity, Kuala Lumpur",1800000,3+1,3.0,NaN,Condominium,Land area : Kuala Lumpur sq. ft.,Partly Furnished
1183,"Desa ParkCity, Kuala Lumpur",2800000,3+1,4.0,NaN,3-sty Terrace/Link House (Intermediate),Land area : Kuala Lumpur sq. ft.,Partly Furnished
2506,"Pantai, Kuala Lumpur",5500000,4+1,5.0,NaN,Bungalow,Land area : Malaysia sq. ft.,NaN
3319,"KLCC, Kuala Lumpur",1400000,Studio,1.0,NaN,Serviced Residence (Intermediate),Land area : - sq. ft.,Partly Furnished


In [19]:
# get the count of rows where the Size column has no digits
df_train[~df_train.Size.str.contains(r'\d')].shape

(50, 8)

In [20]:
def clean_up_size(df_train,clean_up_list):
    '''
    For the Size column:
    - lowercase values 
    - split the dimension values from the size type values
    - remove records with no numeric values
    - remove records with problematic values in clean_up_list
    '''

    # lowercase values in the Size column
    df_train['Size'] = df_train['Size'].str.lower()

    # split the Size column into two columns
    df_train[['Size_type','Size']] = df_train['Size'].str.split(':',expand=True)

    # replace missing values in the Size column with 0
    df_train['Size'] = df_train['Size'].fillna("0")
    
    # remove rows from Size that do not contain any digits
    df_train = df_train[df_train.Size.str.contains(r'\d')]
    # remove rows from Size that contain substrings from the clean_up_list
    for string in clean_up_list:
        df_train = df_train[~df_train.Size.str.contains(string,na=False)]
    # replace extraneous characters so that all Size entries are either numeric or of the form "numerica * numericb"
    df_train['Size'] = df_train['Size'].str.replace(',','').str.replace('`','').str.replace('@','x').str.replace('\+ sq. ft.','')
    df_train['Size'] = df_train['Size'].str.replace(' sq. ft.','').str.replace('sf sq. ft.','').str.replace('ft','').str.replace('sq','').str.replace("xx","*").str.replace("x ","*").str.replace(" x","*").str.replace("x","*").str.replace("X","*").replace('\'','')
    # remove extraneous characters following spaces
    df_train['Size'] = df_train['Size'].apply(lambda x: remove_after_space(x))
    # apply transformation to replace "numerica * numericb" with the result of the multiplication
    df_train['Size'] = df_train['Size'].apply(lambda x: eval(str(x)))
    return(df_train)

In [21]:
# call the function to perform all the clean up steps on the size column
clean_up_list = ["-","\+",'\'','\~',"xx","sf","acre","#"]
df_train = clean_up_size(df_train,clean_up_list)

In [22]:
# get the record count after the Size column has been cleaned up
df_train.shape

(52309, 9)

In [23]:
df_train.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Size_type
0,"KLCC, Kuala Lumpur",1250000,2+1,3.0,2.0,Serviced Residence,1335.0,Fully Furnished,built-up
1,"Damansara Heights, Kuala Lumpur",6800000,6,7.0,NaN,Bungalow,6900.0,Partly Furnished,land area
2,"Dutamas, Kuala Lumpur",1030000,3,4.0,2.0,Condominium (Corner),1875.0,Partly Furnished,built-up
4,"Bukit Jalil, Kuala Lumpur",900000,4+1,3.0,2.0,Condominium (Corner),1513.0,Partly Furnished,built-up
5,"Taman Tun Dr Ismail, Kuala Lumpur",5350000,4+2,5.0,4.0,Bungalow,7200.0,Partly Furnished,land area


# Check for missing values

In [24]:
# get a count by column of missing values
count = df_train.isna().sum()
df_train_missing = (pd.concat([count.rename('missing_count'),
                     count.div(len(df_train))
                          .rename('missing_ratio')],axis = 1)
             .loc[count.ne(0)])

In [25]:
df_train_missing

,missing_count,missing_ratio
Rooms,1521,0.029077
Bathrooms,1810,0.034602
Car Parks,16962,0.324265
Furnishing,6471,0.123707


In [26]:
df_train_missing.shape

(4, 2)

# Set the target
Adjust the Price column for binary classification

In [27]:
# function to replace target values with value indicating whether the input is over or under the mean
# note that setting the target to be a string like this results in much higher accuracy (94%) vs. setting
# the target to be a float (accuracy ~ 76%)
def under_over(x,mean_x):
    if (x <= mean_x):
        #returner = 0.0
        returner = "0"
    else:
        returner = "1"
    return(returner)

In [28]:
# set target column
mean_sp = int(df_train['Price'].mean())
if categorical_target:
    df_train['Price'] = df_train['Price'].apply(lambda x: under_over(x,mean_sp))
df_train.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Size_type
0,"KLCC, Kuala Lumpur",0,2+1,3.0,2.0,Serviced Residence,1335.0,Fully Furnished,built-up
1,"Damansara Heights, Kuala Lumpur",1,6,7.0,NaN,Bungalow,6900.0,Partly Furnished,land area
2,"Dutamas, Kuala Lumpur",0,3,4.0,2.0,Condominium (Corner),1875.0,Partly Furnished,built-up
4,"Bukit Jalil, Kuala Lumpur",0,4+1,3.0,2.0,Condominium (Corner),1513.0,Partly Furnished,built-up
5,"Taman Tun Dr Ismail, Kuala Lumpur",1,4+2,5.0,4.0,Bungalow,7200.0,Partly Furnished,land area


In [29]:
mean_sp

1977360

In [30]:
# check the proportion of Price values
df_train['Price'].value_counts()

0    39018
1    13291
Name: Price, dtype: int64

In [31]:
df_train.shape

(52309, 9)

# Define the target, continuous and categorical columns

In [32]:
# define transforms to apply to the tabular dataset
procs = [FillMissing,Categorify]
# define the dependent variable (y value)
dep_var = 'Price'
# define columns that are continuous / categorical
cont,cat = cont_cat_split(df_train, 1, dep_var=dep_var) 
print("continuous columns are: ",cont)
print("categorical columns are: ",cat)

continuous columns are:  ['Bathrooms', 'Car Parks', 'Size']
categorical columns are:  ['Location', 'Rooms', 'Property Type', 'Furnishing', 'Size_type']


# define TabularDataLoaders

In [33]:
# define TabularDataLoaders object using the dataframe, the list of pre-processing steps, the categorical and continuous
# column lists
# valid_idx: the indices to use for the validation set
procs = [FillMissing,Categorify, Normalize]
dls = TabularDataLoaders.from_df(df_train,path,procs= procs, 
                                 cat_names= cat, cont_names = cont, 
                                 y_names = dep_var, 
                                 valid_idx=list(range((df_train.shape[0]-5000),df_train.shape[0])), bs=64)
                               

In [34]:
# display a sample batch
dls.valid.show_batch()

,Location,Rooms,Property Type,Furnishing,Size_type,Bathrooms_na,Car Parks_na,Bathrooms,Car Parks,Size,Price
0,"KL Sentral, Kuala Lumpur",1,Serviced Residence,Fully Furnished,built-up,False,False,1.0,1.0,1119.000007,0
1,"Bangsar, Kuala Lumpur",2,Serviced Residence,Fully Furnished,built-up,False,False,2.0,1.0,1076.000009,0
2,"KLCC, Kuala Lumpur",3,Serviced Residence,#na#,built-up,False,True,3.0,2.0,1799.999996,1
3,"KLCC, Kuala Lumpur",Studio,Serviced Residence,Fully Furnished,built-up,False,False,1.0,1.0,4491.000025,0
4,"KLCC, Kuala Lumpur",3+1,Condominium,#na#,built-up,False,False,4.0,2.0,2614.999989,1
5,"KLCC, Kuala Lumpur",5+2,Condominium,Fully Furnished,built-up,False,True,7.0,2.0,7199.999873,1
6,"KLCC, Kuala Lumpur",2,Condominium (Corner),Partly Furnished,built-up,False,False,2.0,1.0,1624.000035,1
7,"Sri Petaling, Kuala Lumpur",3,Condominium,Partly Furnished,built-up,False,False,2.0,1.0,1199.999967,0
8,"Wangsa Maju, Kuala Lumpur",3,Condominium (Intermediate),Partly Furnished,built-up,False,False,2.0,1.0,1149.999996,0
9,"Cheras, Kuala Lumpur",5+1,3-sty Terrace/Link House (Intermediate),Partly Furnished,land area,False,False,5.0,2.0,23.999925,0


In [35]:
# define and fit the model
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.175031,0.174658,0.929600,00:14
1,0.143417,0.147668,0.936000,00:14
2,0.128981,0.146698,0.938600,00:14


In [36]:
# show the loss function used by the learner
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [37]:
# show a set of results from the model
learn.show_results()

,Location,Rooms,Property Type,Furnishing,Size_type,Bathrooms_na,Car Parks_na,Bathrooms,Car Parks,Size,Price,Price_pred
0,3.0,0.0,56.0,4.0,2.0,2.0,2.0,-0.053581,-0.010860,0.245799,1.0,1.0
1,14.0,1.0,88.0,2.0,1.0,1.0,1.0,-1.299513,-0.931626,-0.042105,0.0,0.0
2,56.0,2.0,88.0,2.0,1.0,1.0,1.0,-1.299513,-0.931626,-0.027924,0.0,0.0
3,89.0,27.0,76.0,0.0,2.0,2.0,2.0,-0.053581,-0.010860,0.947749,1.0,1.0
4,25.0,0.0,74.0,0.0,2.0,2.0,2.0,-0.053581,-0.010860,0.150699,1.0,1.0
5,19.0,24.0,51.0,2.0,2.0,1.0,2.0,0.569386,-0.010860,0.525406,1.0,1.0
6,14.0,18.0,85.0,1.0,1.0,1.0,2.0,-0.053581,-0.010860,-0.026419,1.0,1.0
7,45.0,18.0,84.0,1.0,1.0,1.0,1.0,-0.676547,-0.931626,-0.035274,0.0,0.0
8,56.0,1.0,84.0,0.0,1.0,1.0,1.0,-0.676547,-0.931626,-0.039847,0.0,0.0


# Examine the structure of the trained model structure

Use the summary() function to see the structure of the trained model, including:

- the layers that make up the model
- total parameters
- loss function
- optimizer function
- callbacks

In [38]:
learn.summary()

TabularModel (Input shape: ['64 x 7', '64 x 3'])
Layer (type)         Output Shape         Param #    Trainable 
Embedding            64 x 22              2,420      True      
________________________________________________________________
Embedding            64 x 13              572        True      
________________________________________________________________
Embedding            64 x 21              2,058      True      
________________________________________________________________
Embedding            64 x 4               20         True      
________________________________________________________________
Embedding            64 x 3               9          True      
________________________________________________________________
Embedding            64 x 3               9          True      
________________________________________________________________
Embedding            64 x 3               9          True      
_________________________________________________